In [ ]:
# Lectura del archivo .csv de training
import pyspark_csv as pycsv
sc.addPyFile('pyspark_csv.py')
plaintext_rdd = sc.textFile('muestra100.csv')
dataframe = pycsv.csvToDataFrame(sqlCtx, plaintext_rdd, parseDate=False)


data = dataframe.rdd
data

In [ ]:
# Esta celda ya expiró (ni mirar)
from sklearn.feature_extraction import DictVectorizer
from MulticatSVM import *
import numpy as np

def parsearReview(x):
    parser = SymbolParser('palabra')
    texto = x[0]
    tabla = x[1]
    parser.parsearTexto(texto, tabla)
    return tabla

def mergeSymbolTables(tablaA, tablaB):
    # Copio A en C
    tablaC = SymbolTable()
    for simb in tablaA.verSimbolos():
        freq = tablaA.verFrecuencia(simb)
        while (freq > 1):
            tablaC.aumentarFrecuencia(simb)
            freq -= 1  
    # Veo ahora los simbolos en B
    for simb in tablaB.verSimbolos():
        freq = tablaB.verFrecuencia(simb)
        while (freq > 1):
            tablaC.aumentarFrecuencia(simb)
            freq -= 1    
    return tablaC

def vectorizar_reviews(elSet):
    tablas = elSet.map(lambda x: (x[0], SymbolTable())).map(parsearReview)
    tablas = tablas.map(lambda x: dict(x.verItems())).collect()


reviews = data.map(lambda x: x.Text)

desayunos = ["cafe medialunas torta", "cafe cafe te", "medialunas torta medialunas", 
     "te torta", "cafe medialunas medialunas", "te torta torta", "cafe torta"]
almuerzos = ["fideos carne", "milanesa carne", "fideos milanesa", "arroz milanesa",
            "fideos milanesa carne", "carne arroz arroz", "fideos fideos", "milanesa"]
reviews = sc.parallelize(desayunos, 4) # Matar post ejemplo
tablas = reviews.map(lambda x: (x, SymbolTable())).map(parsearReview)
tablas = tablas.map(lambda x: dict(x.verItems())).collect()
#print tablas
vectorizador = DictVectorizer(sparse=False)
desayuno_vectorizado = vectorizador.fit_transform(tablas)
pepe = []
for v in range (0, len(desayuno_vectorizado)):
    pepe.append(np.append(desayuno_vectorizado[v], np.array([0.0, 0.0, 0.0, 0.0])))
desayuno_vectorizado = pepe

print desayuno_vectorizado

# Matar después:
# Desayuno: categoría 1; Almuerzo: categoría 2
reviews = sc.parallelize(almuerzos, 4) # Matar post ejemplo
tablas = reviews.map(lambda x: (x, SymbolTable())).map(parsearReview)
tablas = tablas.map(lambda x: dict(x.verItems())).collect() 
vectorizador = DictVectorizer(sparse=False)
almuerzo_vectorizado = vectorizador.fit_transform(tablas)
pepe = []
for v in range (0, len(almuerzo_vectorizado)):
    pepe.append(np.append(np.array([0.0, 0.0, 0.0, 0.0]), almuerzo_vectorizado[v]))
almuerzo_vectorizado = pepe
print almuerzo_vectorizado

comidas = []
comidas.append(desayuno_vectorizado)
comidas.append(almuerzo_vectorizado)

our_svm = MulticatSVM(dim = 8, cte_soft_margin = 9000.0, cantCategorias = 2)
our_svm.entrenar(comidas)

print "Categoria desayuno:", our_svm.predecir(np.array([1., 1., 0., 2., 0., 0., 0., 0.]))
print "Categoria almuerzo:", our_svm.predecir(np.array([0., 0., 0., 0., 1., 0., 2., 0.]))

In [ ]:
from compresores import *


# cmpr es una lista de compresores estáticos, donde el compresor i
# es encargado de comprimir reviews de puntaje i+1
cmpr = []
for i in range(0,5):
    cmpr.append(CompresorAritmetico('letra'))  
        
# Del set de datos, obtengo para compresor i las reviews de
# puntaje i+1 y las parseo para actualizar las tablas de símbolos
for i in range(0,5):
    reviews = data.filter(lambda x: x.Prediction == (i+1)).map(lambda x: x.Text)
    reviews = reviews.map(lambda x: (x, SymbolTable())).map(parsearReview)
    # Ahora en reviews hay tablas de símbolos que representan
    # las reviews originales. ¡Las mergeamos!
    tablaFinal = reviews.reduce(mergeSymbolTables)
    cmpr[i].asignarSymbolTable(tablaFinal)

# Pruebo mostrando los simbolos de cada compresor
for i in range(0,5):
    print 'Símbolos del conversor', i, ':', cmpr[i].getSymbolTable().verSimbolos()
    print 'Frecuencia de la a:', cmpr[i].getSymbolTable().verFrecuencia('a')


In [ ]:
# DE ACA PARA ABAJO ZONA DE TESTS
# NO DAR MUCHA BOLA



print lasReviews.takeOrdered(1)[0][1]

In [ ]:
q = lasReviews.map(lambda x: (x[0],x[1].count("!!! "))).reduceByKey(lambda x, y: x+y)
print q.collect()

In [ ]:
from enchant.checker import SpellChecker
chkr = SpellChecker("en_US")

reviewsAlAzar = textos.takeSample(False, 5)


for r in range(0,5):
    print reviewsAlAzar[r]
    chkr.set_text(reviewsAlAzar[r])
    for err in chkr:
        print "ERROR! Wrong word:", err.word

In [ ]:
# Todo este bloque define la realización del k-fold crossed validation.
# El método funciona así: recibe un set de entrenamiento y hace sobre el
# mismo la técnica de k-fold crossed validation. El formato del set debe
# ser un RDD de TUPLAS de la forma: (features, categoria) donde la clave
# features puede ser cualquier basura, y categoria es el valor numérico
# que se desea predecir (aka el puntaje de cada review). En cada pasada
# del k-fold crossed validation, se invocan a las funciones de entrenar
# func_entrenar y a las de predicción func_predecir. Éstas dos funciones
# deben trabajar de manera global con el/los compresor/es o el SVM. Sus
# firmas deben ser las siguientes:
# func_entrenar recibe un set de entrenamiento (en el mismo formato que
# el set original, como tuplas feature,cat.) y prepara al compresor o SVM
# para las predicciones usando ese set.
# func_predecir recibe un set a predecir (en el mismo formato de tuplas
# feature, cat) y debe devolver OTRO set (también en el mismo formato!)
# que correspondan a las predicciones hechas por el SVM o compresores.
# Observación importante: como el k-fold crossed validation en sí no
# tiene ni idea qué usamos para predecir, todo lo demás ajeno a eso,
# incluyendo la selección de hiperparametros, debe hacerse "por fuera",
# ya sea con un pre-procesamiento de las reviews o en la función de entrenar.

def calculo_ECM(predSet, valSet):
    cant = predSet.count()
    setAux = predSet.union(valSet)
    setAux = setAux.reduceByKey(lambda x,y: float(x)-float(y)).map(lambda x: x[1]*x[1])
    ecm = setAux.reduce(lambda x,y: x+y)
    return (ecm/float(cant))

def k_fold_crossed_validation(elSet, func_entrenar, func_predecir):
    cantParticiones = 10
    ecm_acum = 0.0
    largoSet = elSet.count()
    largoParticion = largoSet / cantParticiones
    for j in range (1, cantParticiones+1):
        # Obtengo el testSet como la particion j-ésima y el trainSet como
        # todo el resto del set recibido menos el testSet
        testSet = sc.parallelize(elSet.take(largoParticion * j), 4)
        testSet = testSet.subtract(sc.parallelize(elSet.take(largoParticion * (j-1)), 4))
        trainSet = elSet.subtract(testSet)
        # Entreno contra trainSet
        func_entrenar(trainSet)
        # Testeo contra testSet
        setResultados = func_predecir(testSet)
        ecm_acum += calculo_ECM(setResultados, testSet)
        print "ECM acumulado iteracion", j, "es:", ecm_acum
    # Obtengo el ECM promedio de la validación
    print "ECM promedio:", (ecm_acum/float(cantParticiones))

In [ ]:
import numpy as np

def parsearReview(x):
    parser = SymbolParser('palabra')
    texto = x[0]
    tabla = x[1]
    parser.parsearTexto(texto, tabla)
    return tabla

# Método auxiliar usado en el map-reduce: Fusiona dos diccionarios
def unirDiccionarios(dicA, dicB):
    # Copio A en C
    dicC = {}
    for clave in dicA.keys():
        valor = dicA[clave]
        dicC[clave] = valor
    # Veo ahora los simbolos en B
    for clave in dicB.keys():
        valor = dicB[clave]
        if dicC.has_key(clave):
            dicC[clave] += valor
        else:
            dicC[clave] = valor
    return dicC

# Método auxiliar de vectorización: Genera un numpy.vector que representa
# la cant. de palabras de un determinado review.
def vectorizar(x):
    global tabla_vec
    words = tabla_vec.keys()
    p = []
    for w in words:
        if(x.has_key(w)):
            p.append(float(x[w]))
        else:
            p.append(0.0)
    return np.array(p)

# vectorizar_reviews es la función encargada de convertir en numpy.vector
# todos los textos de todas las reviews. Para ello, recibe como parámetro
# el set de entrenamiento, que debe tener el sge formato: debe ser un RDD
# formado por tuplas (texto, puntaje) de cada review. El método devuelve
# otro RDD con el formato (vector, puntaje) donde vector es un numpy.vector
# que representa el texto de cada review recibida. Cada vector se consigue
# de la sgte forma: Primero se listan todas las palabras de todas las
# reviews del set, y se les asigna a cada una de ellas un índice del vector; 
# luego, para cada review del set, se cuentan cuántas palabras hay y cuáles,
# y se ponen esos valores de contadores en las posiciones correspondientes
# del vector. Por ejemplo, si todas las palabras son ["Casa", "Pez", "Arbol"]
# una review de la forma "Casa Pez Casa" se traducirá como el vector (2, 1, 0)
# NOTA: Si se quiere que en vez de contar las palabras se cargue sólamente un
# 1 o un 0 según si la palabra está presente o no, se debe cambiar en la
# función vectorizar de arriba el p.append(float(x[w])) por un p.append(1.0)
# ADVERTENCIA: El vectorizador no hace ningún procesamiento del texto, y
# separa sólo las palabras por espacios. Cualquier pre-procesamiento (por
# ejemplo, eliminar las ",") debe hacerse antes de llamarlo.

def vectorizar_reviews(elSet):
    # Primero consigo en tabla_vec un diccionario con todas las palabras
    # de todas las reviews del set
    global tabla_vec
    tabla_vec = elSet.map(lambda x: (x[0], SymbolTable())).map(parsearReview)
    tabla_vec = tabla_vec.map(lambda x: dict(x.verItems()))
    tabla_vec = tabla_vec.reduce(unirDiccionarios)
    # Vectorizo ahora todas las reviews del set usando esa tabla
    reviews_vec = reviews.map(lambda x: (x[0], SymbolTable(), x[1]))
    reviews_vec = reviews_vec.map(lambda x: (parsearReview(x), x[2]))
    reviews_vec = reviews_vec.map(lambda x: (dict(x[0].verItems()), x[1]))
    reviews_vec = reviews_vec.map(lambda x: (vectorizar(x[0]), x[1]))
    return reviews_vec

In [ ]:
from bs4 import BeautifulSoup # Para eliminar tags html
import re # Expresiones regulares para eliminar puntuacion
from nltk.corpus import stopwords # Stopwords para eliminar palabras comunes
from nltk.stem.lancaster import LancasterStemmer

def aplicarStemming(x):
    words = x.split()
    st = LancasterStemmer()
    new_words = []
    for w in words:
        new_words.append(st.stem(w))
    return " ".join(new_words)

def borrarPalabrasComunes(x):
    words = x.split()
    stop_words = set(stopwords.words("english"))
    stop_words.remove("not")
    new_words = []
    for w in words:
        if(not w in stop_words):
            new_words.append(st.stem(w))
    return " ".join(new_words)

def borrarSimbolos(x):
    aBorrar = ",@#$-.():[]!?"
    for c in aBorrar:
        x = x.replace(c, " ")
    return x

def considerarEmoticonesPuntuacion(x):
    # Lista de caritas felices
    caras_felices = [":)", "(:", "[:", ":]", "c:", "=)", "=]", "(=", "[=", "c=",
                    "=D", ":D", ";)", "(;", ";D"]
    for emoji in caras_felices:
        x = x.replace(emoji , "SMILING_FACE")
    # Lista de caritas tristes
    caras_tristes = [":(", ":[", "):", "]:", ":c", "=(", "=[", "]=", "=c", "D=", 
                    "D:", ";(", ");", "D;", ]
    for emoji in caras_tristes:
        x = x.replace(emoji, "SAD_FACE")
    # Lista de caritas sorprendidas
    caras_sorpr = [":0", ":o", "0:", "o:", "=o", "0="]
    for emoji in caras_sorpr:
        x = x.replace(emoji, "SURPRISED_FACE") 
    # Puntuación (signos ! y ?)
    x = x.replace("!!!", " ADMIRx3")
    x = x.replace("!!", " ADMIRx2")
    x = x.replace("???", " QUESx3")
    x = x.replace("??", " QUESx2")
    x = x.replace("?!", " ADM_QUES")
    x = x.replace("!?", " ADM_QUES")
    x = x.replace("!", " ADMIRx1")
    x = x.replace("?", " QUESx1")
    return x

# Función encargada de realizar un pre-procesamiento de los textos de las reviews
# según lo considerado por nuestro diseño del TP. Para ello, se recibe el set de
# entrenamiento como un RDD de reviews, que son tuplas (texto, puntaje).
# Las distintas acciones que la función realiza sobre el texto de las reviews
# dependen de los flags de procesamiento recibidos en flagsP (como lista).
# A continuación la lista de acciones controlada por cada flag de flagsP:
# flagsP[0] controla la eliminación de palabras comunes ("a", "the", "of", etc.)
# fragsP[1] elimina las palabras de frecuencia menor a *un número*
# flagsP[2] activa el uso de stemming sobre las palabras de la review
# flagsP[3] activa el reconocimiento de emoticones y puntuaciones ?,!
# continuar...
# Acciones que el pre-procesador de reviews hace siempre:
# - Eliminar tags html
# - Convertir todo a minúsculas
# - Eliminar los siguientes símbolos: "," "@" "#" "$" "-" "." "(" ")" ":" "]" "["
# (En el caso de considerar emoticones o puntuación no lo hace hasta después de
# detectar todos los emoticones o símbolos deseados correspondientes)
def preprocesar_reviews(elSet, flagsP):
    nuevoSet = elSet.map(lambda x: (BeautifulSoup(x[0], "lxml").getText(), x[1]) )
    nuevoSet = nuevoSet = nuevoSet.map(lambda x: (x[0].lower(), x[1]))
    
    if(flagsP[0]):
        nuevoSet = nuevoSet.map(lambda x: (borrarPalabrasComunes(x[0]), x[1]))
    
    if(flagsP[2]):
        nuevoSet = nuevoSet.map(lambda x: (aplicarStemming(x[0]), x[1]))
    
    if(flagsP[2]):
        nuevoSet = nuevoSet.map(lambda x: (considerarEmoticonesPuntuacion(x[0]), x[1]))
    nuevoSet = nuevoSet = nuevoSet.map(lambda x: (borrarSimbolos(x[0]), x[1]))
    
    
    return nuevoSet

reviews = data.map(lambda x: (x.Text, x.Prediction))
print reviews.count()
reviews = preprocesar_reviews(reviews, [1, 0, 1, 1])
print reviews.count()
print reviews.take(10)

In [ ]:
from nltk.stem import SnowballStemmer
st = SnowballStemmer("english")
cade = 'maximum provision for our maxim northern allies'
print cade
for w in cade.split():
    print st.stem(w)
print cade

In [65]:
import numpy as np
from scipy import spatial
import time
import math

def kernel_gaussiano(x, y):
        s = 0.3
        numE = 2.7182818284590452
        dist = spatial.distance.sqeuclidean(x, y)
        return numE**((-0.5 * dist) / (s * s) ) 
    
def kernel_gaussiano2(x, y):
        pe = -5.55555555555555556
        numE = 2.7182818284590452
        dist = spatial.distance.sqeuclidean(x, y)
        return numE**(pe * dist) 
    
tAcum = 0
for i in range(0, 20):    
    xx = np.random.randn(25)
    yy = np.random.randn(25)
    t1 = time.time()
    kernel_gaussiano2(xx, yy)
    t2 = time.time()
    deltaT = t2 - t1
    tAcum += deltaT
    print "Iteracion", i+1, ":", deltaT
print "Promedio:", tAcum/20

Iteracion 1 : 0.000102043151855
Iteracion 2 : 6.103515625e-05
Iteracion 3 : 4.19616699219e-05
Iteracion 4 : 4.60147857666e-05
Iteracion 5 : 3.81469726562e-05
Iteracion 6 : 6.38961791992e-05
Iteracion 7 : 4.10079956055e-05
Iteracion 8 : 3.69548797607e-05
Iteracion 9 : 0.000159025192261
Iteracion 10 : 3.81469726562e-05
Iteracion 11 : 3.79085540771e-05
Iteracion 12 : 9.79900360107e-05
Iteracion 13 : 3.88622283936e-05
Iteracion 14 : 3.71932983398e-05
Iteracion 15 : 4.41074371338e-05
Iteracion 16 : 3.91006469727e-05
Iteracion 17 : 3.60012054443e-05
Iteracion 18 : 7.31945037842e-05
Iteracion 19 : 0.000108003616333
Iteracion 20 : 5.07831573486e-05
Promedio: 5.95688819885e-05


In [43]:
from scipy import spatial
import numpy

v = numpy.array([-1.0, 2.0])
w = numpy.array([3.0, 5.0])

print spatial.distance.sqeuclidean(w, v)

25.0


In [54]:
print 0.5/(0.3*0.3)

5.55555555556
